# Sarcasm Detection on Social Media Conversations with BERT
This notebook will show you how to implement a sarcasm classification model with BERT.

We wanted to call this model DilBERT. We know how you feel... 
![alt text](data/sarcastic.gif)
 


## Setup
Make sure you git clone the bert repo inside the root of this repo, and have all the libraries references in environment.yml installed. The syspath addition here will make sure all import references within the BERT repo work.

Ideally dont reload imports, but its good if we want to play around with some defaults which we generally don't touch

In [15]:
import sys
if not 'bert_repo' in sys.path:
    sys.path += ['bert_repo']
    
import importlib
import preprocess
importlib.reload(preprocess)
from preprocess import load_data, standard_loader
import model
importlib.reload(model)
from model import SarcasmBertBasic

## Data
Data loaded can either be twitter, reddit data or both. We can consider all sentences in context or just the most recent context sentence. We can also consider processing emojis to their literal text translation. Emprically, this hasnt helped.  

Another potential improvement is to use a better tokenizer which can recognize misspelling and emojis as part of the tokenization process. This is suggested in the baseline paper. 

## Model
The addition of extra layers to the model is defined in ```SarcasmBertBasic.create_model()```. 
The general structure of the code follows the estimator design pattern. This [paper](https://arxiv.org/abs/1708.02637) articulates best why this is an effective way to develop models.  

```SarcasmBertBasic.fit()``` implements processing the data into what BERT expects using an input_fn builder, the estimator which is finally used to finetune the model, and all the bells and whistles to store models to a local checkpoint. The official [kernel](https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb?hl=fr) explains these steps fairly well.  

```SarcasmBertBasic.predict()``` implements taking the trained estimator and generating predictions  

## Tuning
Since BERT finetuning allows all weights of the Base model to be altered, it is prone to [catastrophic forgetting](https://arxiv.org/pdf/1905.05583.pdf), and we stick to the hyperparameter recommendations made by the BERT authors, after facing a few catastorphes ourselves. In our experiment with this dataset we found it most useful to increase Max Sequence Length to 512, keeping the recommended batch size. Number of Epochs seems to be the tuning parameter that really yields the most value in tuning.  

## Hardware
This was trained on a V100 GPU on Google Cloud - #Credits


In [11]:
print("Loading Data")
train, test = load_data(load_fn=standard_loader('twitter'), context_extent='all', split=0.2)
print("Init")
cfg = {
    'model_ver': 'v1subtest',
    'BERT_MODEL': 'uncased_L-12_H-768_A-12',
    'NUM_TRAIN_EPOCHS': 3.0,
}
m = SarcasmBertBasic(cfg)
print("Begin Train")
m.fit(train)

Loading Data
Init
***** Model output directory: bert_out_files/v1subtest *****
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Begin Train
INFO:tensorflow:Using config: {'_model_dir': 'bert_out_files/v1subtest', '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0e94035c50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'bert_out_files/v1subtest', '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0e94035c50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Writing example 0 of 4000


...

***** Started training at 2020-03-15 07:11:27.349532 *****
  Num examples = 4000
  Batch size = 32
INFO:tensorflow:  Num steps = 375


INFO:tensorflow:  Num steps = 375


INFO:tensorflow:Skipping training since max_steps has already saved.


INFO:tensorflow:Skipping training since max_steps has already saved.


***** Finished training at 2020-03-15 07:11:27.363194 *****


In [12]:
import numpy as np
preds = m.predict(test)

y_true = [e.label for e in test]
labels = [ 'NOT_SARCASM', 'SARCASM']
y_pred = []

# Dont try and be cute by vectorizing this. It runs out of memory. 
for p in preds:
    log_probs = p['probabilities']
    probs = [np.exp(e) for e in log_probs]
    lab = labels[np.argmax(probs)]
    y_pred.append(lab)
    
print("DONE.")

INFO:tensorflow:Writing example 0 of 1000


INFO:tensorflow:Writing example 0 of 1000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 1


...

INFO:tensorflow:Restoring parameters from bert_out_files/v1subtest/model.ckpt-375


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


## Evauation
Manually evaluate our model for now. We should ideally use our ```estimator.train_and_evaluate()``` to monitor at which epoch/step training and test accuracy start diverging, since performance is very sentitive to number of epochs (even between epoch 2 and 3)  

Even though we see high DEV accuracies, they do drop in our test evaluation. To really evaluate the nature of our performance, we need to perform some systematic error analysis as was done in the baseline paper. Did we catch the longer sentences? Are they truly diffcult sarcastic texts? Are there many references to Named entities and world knowledge concepts? Are we doing poor on emoji filled responses. We will release such analysis on this model. 

## Example Misclassifications
**Case 1: Complex reference to a player not making it to the team. Very subtle Sarcasm.**  
response: Do you think Perez will make the 18 this year?  
Context: Could this be the La Liga player we're going after? It's clearly the return of Victor Perez.  
label:SARCASM  
prediction:[0.9902841  0.00971597] NOT_SARCASM  
  
**Case 2: Hypothetical modificaion of sentence to mock by mimicking immediately recognized.**   
Response: After that last match Perez is clearly no victor  
Context: Could this be the La Liga player we're going after? It's clearly the return of Victor Perez.  
label:SARCASM  
prediction:[0.05655888 0.94344115]  SARCASM



In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

acc = accuracy_score(y_true,y_pred)
prec = precision_score(y_true,y_pred,pos_label='SARCASM')
rec = recall_score(y_true,y_pred,pos_label='SARCASM')
f1 = f1_score(y_true,y_pred,pos_label='SARCASM')

ns_prec = precision_score(y_true,y_pred,pos_label='NOT_SARCASM')
ns_rec = recall_score(y_true,y_pred,pos_label='NOT_SARCASM')
ns_f1 = f1_score(y_true,y_pred,pos_label='NOT_SARCASM')

print( 
    """Accuracy:{}\n\nSARCASM\nPrecision:{}\nRecall:{}\nF1 Score:{}
    \nNOT SARCASM\nPrecision:{}\nRecall:{}\nF1 Score:{}""".format(acc,prec,rec,f1,ns_prec,ns_rec,ns_f1)
)

Accuracy:0.803

SARCASM
Precision:0.835920177383592
Recall:0.754
F1 Score:0.7928496319663513
    
NOT SARCASM
Precision:0.7759562841530054
Recall:0.852
F1 Score:0.8122020972354624


## Competition
Some post processing of predictions to submit our results to the CodaLab Shared task. 

In [16]:
import pandas as pd
# Generate final results for Live Test Data set
live_test = load_data(load_fn=standard_loader('twitter_test'), context_extent='all', split=None)
live_df = load_data(load_fn=standard_loader('twitter_test'),bert_pair_shape=False, context_extent='all',split=None)

ids = live_df['id'].tolist()

live_preds = m.predict(live_test)
labels = [ 'NOT_SARCASM', 'SARCASM']
#live_y_pred = []

# Dont try and be cute by vectorizing this. It runs out of memory. 
final_res = pd.DataFrame()
for this_id, p in zip( ids, live_preds):
    log_probs = p['probabilities']
    probs = [np.exp(e) for e in log_probs]
    lab = labels[np.argmax(probs)]
    #live_y_pred.append(lab)    
    final_res = final_res.append({'label': lab, 'id': this_id },ignore_index=True)

INFO:tensorflow:Writing example 0 of 1800


INFO:tensorflow:Writing example 0 of 1800


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 1


...

INFO:tensorflow:Restoring parameters from bert_out_files/v1subtest/model.ckpt-375


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [4]:
f = open("answer.txt", "w")

for _ind, row in final_res.iterrows():
    f.write('{},{}\n'.format(row['id'],row['label']))
    
f.close()